<center> <h1> Anaylze the order of dyes bleaching reaction</h1> </center>

### Overview
1. [Introduction](#1.-Introduction)
2. [Minimal Background for Python](#2.-Minimal-Background-for-Python)
3. [Data Analysis](#3.-Data-Analysis)

---

## 1. Introduction

In order to determine the rate law for a reaction from a set of data consisting of concentration (or the values of some function of concentration) versus time, make three graphs.

- [A] versus t (linear for a zero order reaction)
- ln [A] versus t (linear for a 1st order reaction)
- 1 / [A] versus t (linear for a 2nd order reaction)

The graph that is linear indicates the order of the reaction with respect to A. Then, you can choose the correct rate equation:

For zero-order reactions: $[A]_t = −kt + [A]_0$

For first-order reactions: $ln[A]_t = −kt + ln[A]_0$

For second-order reactions: $1/[A]_t = kt + 1/[A]_0$

---

## 2. [Minimal Background for Python](https://yyrcd-1256568788.cos.na-siliconvalley.myqcloud.com/yyrcd/2019-10-18-Minimal%20Background%20for%20Python-1.html)

1. Basic data types
2. Containers
3. Control Flow
4. Functions
5. Numpy
6. Matplotlib
7. Scipy


---

## 3. Data Analysis

Your csv should looks like the figure below.   
At least `Latest: Time (min)` column and `Latest: Absorbance at ***.* nm` column must exsit, which contain your experimental data.


![](https://yyrcd-1256568788.cos.na-siliconvalley.myqcloud.com/yyrcd/2019-10-29-163858.png)

In [ ]:
import pandas as pd
import numpy as np
import io
from scipy.optimize import minimize    # fitting tool
from matplotlib import pyplot as plt   # plotting
%matplotlib inline

In [ ]:
def upload():
    from google.colab import files
    uploaded = files.upload()
    csvfile = io.BytesIO(uploaded[list(uploaded.keys())[0]])
    return csvfile

In [ ]:
file = upload()

In [ ]:
def extract_data(file):
    """
    Extract times and concentrations data from one csv file

    Parameters
    -------
    file : location of the txt file
    for example:
    1. uploaded csv file on google.colab
    2. path of csv on your own pc '\home\richard\chm2045L\enzyme.csv'
    3. an url to the csv file

    Returns
    -------
    times : array (sec)
    concentrations : array (mol/L)
    """
    data = pd.read_csv(file)
    headers = list(data.columns)

    for i, d in enumerate(headers):
        if 'Latest: Time' in d:
            pass
            times = data[headers[i]].values
        if 'Latest: Absorbance' in d:
            pass
            absorbances = data[headers[i]].values
    try:
        times
    except NameError:
        print("Error: There must one colum which name contain \"Latest: Time\"")

    try:
        absorbances
    except NameError:
        print("Error: There must one colum which name contain \"Latest: Absorbance\"")

    times = times[np.logical_not(np.isnan(times))]  # remove Nan from the array if Nan exist
    times = times * 60  # convert to secends
    absorbances = absorbances[np.logical_not(np.isnan(absorbances))]  # remove Nan from the array if Nan exist
    concentrations = absorbances / 79600

    return times, concentrations

In [ ]:
def fit_and_plot(times, y_true, label):

    def linear(x, a, b):
        y = a * x + b
        return y

    def loss(para):
        pred = linear(times, para[0], para[1])
        mse = np.mean((y_true - pred)**2)  # mean_squared_error
        mse = mse * 1e8
        return mse

    def r_square(para):
        pred = linear(times, para[0], para[1])
        res = np.sum((y_true - pred)**2)  # residual
        total = np.sum((y_true - np.mean(y_true))**2)
        return 1 - res/total
        
    result = minimize(loss, x0=np.array([1.0, 1.0]), method='Nelder-Mead')
    a = result.x[0]
    b = result.x[1]
    
    plt.figure(figsize=(16, 4))
    plt.plot(times, y_true, 'o-', markersize=1, label=label)
    r2 = r_square([a, b])
    plt.plot(times, linear(times, a, b), 'o-', markersize=1, label='y = {:.3e}x + {:.3e}   $\;\;\;R^2$ = {:.4f}'.format(a, b, r2))
    plt.xlabel('Time (sec)')
    plt.ylabel(label)
    plt.title('{} over Time'.format(label),  fontsize=18)
    plt.legend()
    plt.show()

In [ ]:
times, concentrations = extract_data(file)

In [ ]:
fit_and_plot(times, y_true=concentrations, label='[dyes]')

In [ ]:
fit_and_plot(times, y_true=np.log(concentrations), label='Ln [dyes]')

In [ ]:
fit_and_plot(times, y_true=1 / concentrations, label='1 / [dyes]')